In [14]:
import open3d as o3d
import numpy as np
import os
from tqdm import tqdm
import tensorflow as tf

## Fix your path here

In [15]:
filepath_kri = ["/Users/kristofferkjaer/Desktop/mesh", "/Users/kristofferkjaer/Desktop/SPRING_MALE"]

filepath_karo = ["/Users/KAROLINE/Desktop/Fagprojekt/02466-Fagpakkeprojekt-Point-clouds/Data/Female","/Users/KAROLINE/Desktop/Fagprojekt/02466-Fagpakkeprojekt-Point-clouds/Data/Male"]

In [16]:
data_paths = filepath_kri

N = 5

pcds_females = []
pcds_males = []
pcds = [pcds_females, pcds_males]

for data in range(len(data_paths)):
    files = []
    folder_to_view = data_paths[data]
    list = pcds[data]
    for file in os.listdir(folder_to_view):
        files.append(file)
    for i in tqdm(range(N)):
        file = (folder_to_view + "/" + str(files[i]))
        mesh = o3d.io.read_triangle_mesh(file)
        pcd = mesh.sample_points_poisson_disk(2048)
        list.append(pcd)

100%|██████████| 5/5 [00:02<00:00,  1.94it/s]


### Augment

In [17]:
def augment(points):
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    points = tf.random.shuffle(points)
    return points

### Convert to numpy array

In [18]:
pcds_females_arr = []
pcds_males_arr = []
pcds_arr = [pcds_females_arr, pcds_males_arr]


for points in range(len(pcds_arr)): 
    for i in tqdm(range(len(pcds_males_arr))):
        pcds_arr[points][i].append(np.asarray(pcds_arr[points][i].points) )



0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [26]:
furthest_distance = 0

for points in range(len(pcds_arr)): 
    for i in tqdm(range(len(pcds_males_arr))):
        distance = np.max(np.sqrt(np.sum(abs(points)**2,axis=-1)))
        if distance > furthest_distance:
            furthest_distance = np.max(np.sqrt(np.sum(abs(pcds_arr[points][i])**2,axis=-1)))


furthest_distance

100%|██████████| 5/5 [00:00<00:00, 4659.30it/s]


1.0

### Normalize (Update to normalize in regards to biggest person)

In [27]:

def normalize(points): # The input of this function is a point cloud object
    # This function normalizes a point cloud within a unit sphere
    points = np.asarray(points.points) 
    centroid = np.mean(points, axis=0)
    points -= centroid
    points /= furthest_distance
    return points # The function return a numpy array of dimension nx3 where the points are normalized within a unit sphere

pcds_females_arr_norm = []
pcds_males_arr_norm = []
pcds_arr_norm = [pcds_females_arr, pcds_males_arr]


for points in range(len(pcds_arr)):

    for i in tqdm(range(len(pcds_females))):
        pcds_arr_norm[points].append(normalize(pcds[points][i]))

100%|██████████| 5/5 [00:00<00:00, 592.48it/s]


### To visualise point cloud

In [21]:
#o3d.visualization.draw_geometries([pcds_females[2]])

### Augment

In [22]:
def augment(points):
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    points = tf.random.shuffle(points)
    return points